# Nowcast Experiment (NO VCI in X DATA)

In [2]:
from pathlib import Path
import os
import warnings

%load_ext autoreload
%autoreload

# ignore warnings for now ...
warnings.filterwarnings('ignore')

if Path('.').absolute().parents[1].name == 'ml_drought':
    os.chdir(Path('.').absolute().parents[1])

!pwd

/Users/tommylees/github/ml_drought


In [3]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

data_dir = Path('data/')
# data_dir = Path('/Volumes/Lees_Extend/data/zip_data')
data_dir = Path('/Volumes/Lees_Extend/data/ecmwf_sowc/data/')

assert data_dir.exists()

In [4]:
import seaborn as sns
from src.utils import drop_nans_and_flatten

from src.analysis import read_train_data, read_test_data, read_pred_data

# Read in the data

In [5]:
X_train, y_train = read_train_data(data_dir)
X_test, y_test = read_test_data(data_dir)

In [6]:
ds = xr.merge([y_train, y_test]).sortby('time').sortby('lat')
d_ = xr.merge([X_train, X_test]).sortby('time').sortby('lat')
ds = xr.merge([ds, d_])

In [7]:
from src.utils import get_ds_mask
mask = get_ds_mask(X_train.VCI)

In [8]:
[d.name for d in (data_dir / 'models' / '2020_01_13:151544_nowcast_tommy').iterdir()]

['ealstm', 'linear_network', 'linear_regression', 'previous_month', 'rnn']

In [48]:
# Standard Experiments
bline_pred = read_pred_data('previous_month', data_dir)[-1].where(~mask)
orig_ln_pred = read_pred_data('linear_network', data_dir,)[-1].where(~mask)
orig_rnn_pred = read_pred_data('rnn', data_dir,)[-1].where(~mask)
orig_ealstm_pred = read_pred_data('ealstm', data_dir)[-1].where(~mask)

# Predict Delta Experiments 
ln_pred = read_pred_data('linear_network', data_dir, experiment='one_month_forecast_predict_delta')[-1].where(~mask)
ealstm_pred = read_pred_data('ealstm', data_dir, experiment='one_month_forecast_predict_delta')[-1].where(~mask)

# ealstm_pred = read_pred_data('ealstm', data_dir)

## Explore how perform in each month

In [13]:
from src.analysis import annual_scores

In [23]:
print((data_dir / 'models/one_month_forecast_predict_delta/rnn/').exists())
[d.name for d in (data_dir / 'models/one_month_forecast_predict_delta/rnn/').iterdir()]

True


[]

In [46]:
# load the monthly scores dictionary
experiment = 'one_month_forecast_predict_delta'
monthly_scores = annual_scores(
    data_path=data_dir,
    models=['linear_network', 'ealstm'],
    metrics=['rmse', 'r2'],
    pred_years=[y for y in range(2011, 2019)],
    experiment=experiment,
    true_data_experiment='one_month_forecast',
    target_var='VCI',
    verbose=False,
    to_dataframe=True
)
monthly_scores['time'] = monthly_scores.apply(lambda row: pd.to_datetime(f"{int(row.month)}-{int(row.year)}"), axis=1)

monthly_scores2 = annual_scores(
    data_path=data_dir,
    models=['linear_network', 'ealstm', 'rnn', 'previous_month'],
    metrics=['rmse', 'r2'],
    pred_years=[y for y in range(2011, 2019)],
    experiment='one_month_forecast',
    true_data_experiment='one_month_forecast',
    target_var='VCI',
    verbose=False,
    to_dataframe=True
)
monthly_scores2['time'] = monthly_scores.apply(lambda row: pd.to_datetime(f"{int(row.month)}-{int(row.year)}"), axis=1)

# rename columns and to merge dataframes
monthly_scores2 = monthly_scores2.join(
    monthly_scores2[['linear_network', 'ealstm', 'rnn']].add_prefix('orig_')
).drop(columns=['linear_network', 'ealstm', 'rnn'])

# merge
monthly_scores = pd.merge(monthly_scores, monthly_scores2)
monthly_scores.head()

,ealstm,linear_network,month,year,metric,time,previous_month,orig_linear_network,orig_ealstm,orig_rnn
0,9.518235,20.316086,1.0,2011.0,rmse,2011-01-01,9.79577,22.490418,13.402106,11.853916
1,9.518235,20.316086,1.0,2011.0,rmse,2011-01-01,9.79577,-0.354995,0.518841,0.623585
2,9.518235,20.316086,1.0,2011.0,rmse,2011-01-01,9.79577,22.490418,13.402106,11.853916
3,9.518235,20.316086,1.0,2011.0,rmse,2011-01-01,9.79577,-0.354995,0.518841,0.623585
4,9.518235,20.316086,1.0,2011.0,rmse,2011-01-01,9.79577,22.490418,13.402106,11.853916
